## Large problems: Recipes and EAO strategy

In this notebook we explore large problems (in terms of memory or run-time). We explain the main drivers for size & complexity and describe simplifications we found useful.

EAO generates mathematical problems (LP or MIP) out of given assets or portfolios - then passing them to solvers (such as Gurobi, CPLEX or free solvers such as SCPI or HighS). In that sense it is very similar to other commercial or open source approaches.

No matter what solver (or asset optimization framework) is used, once we model complex assets and/or large time spans, problems may hit memory or run-time limits. **EAO's strategy is not to hide away simplifications** but to keep the exact formulation of the problem. We leave it to the user, deciding where she can live with simplifications. 

In the following we give a **recipe, which levers to pull** to keep problems solvable.

#### Basics - setting things up

In [14]:
import numpy as np
import pandas as pd
import datetime as dt
from copy import deepcopy
import eaopack as eao
import tracemalloc ### trace memory usage
import time 

### load input data (price samples)
data = pd.read_excel("data2024.xlsx")
data.set_index('datum', inplace=True)

In [15]:
## keeping track of peak memory
import psutil # import Python psutil module

def memory_usage():
    process = psutil.Process()
    usage = process.memory_info().rss 
    # Using memory_info() to check consumption
    return usage # Returning the memory in bytes

def memory_used():
    return np.round(tracemalloc.get_traced_memory()[1]/(1024.**2),0)

## Large problems - focus on memory

#### Seemingly "large" problem, that actually isn't

We start with setting up a typical "large" problem:

* Let us optimize across one year - hourly resolution (for demo purpose - arguments also hold for longer periods)
* 20 generation assets in the portfolio (we will start with very simple assets - that means no ramping restrictions or startups)
* plus (later on) a storage
* fixed demand to be covered

In [16]:
### setup 
number_of_generating_assets = 20
Start = dt.datetime(2024,1,1)
End   = dt.datetime(2025,1,1) # one year (end, i.e. 01/01:2025 00:00) is not included)
resolution = 'h'  # hourly resolution

Note that we can easily steer the number of time steps modeled using the resolution (or frequency). For power, prices mostly live an a scale of 15min, 30min or hours and the resolution is needed. For gas or other commodities, days, weeks or months are enough. 

**Recommendation: Always to choose the lowest resolution/ frequency that captures your problem**. Trivial example: modelling a year on hourly level leads to 8.760 time steps, on daily level 365. A big difference! It may be worthwhile to compare hourly, 15min or daily optimization to decide what is enough. May save computation or waiting times.

Let us generate the portfolio. We use a simplistic way of generating generators with random capacity and generation costs - note that we start tracking memory usage

In [17]:
tracemalloc.start() ## start memory tracking
# overall structure
node_power  = eao.assets.Node(name = 'power')

# demand
demand = eao.assets.SimpleContract(name    = 'demand',
                                   nodes   = node_power,
                                   min_cap = "demand_curve",
                                   max_cap = "demand_curve")

all_assets = [demand] # collect for portfolio
# supply - various assets
## using random generation costs
np.random.seed(42)
for i in range(number_of_generating_assets):
    gen = eao.assets.SimpleContract(name    = f'gen_{i}',
                                    nodes   = node_power,
                                    min_cap = 0,
                                    max_cap = np.random.uniform(0.5,2),
                                    price   = f'price_gen_{i}'
                                    )
    all_assets.append(deepcopy(gen))
    # add random prices to input data
    data[f'price_gen_{i}'] = np.random.uniform(5,20,len(data)) 
portf = eao.Portfolio(assets = all_assets)
timegrid = eao.Timegrid(Start, End, freq=resolution)
# stopping memory tracking
print("Peak memory usage: ", f'{memory_used():,.0f}', "MB")
tracemalloc.stop()


Peak memory usage:  2 MB


Setting up the portfolio does not require much memory. Basically little has happened but storing parameters in the objects.

Now: We do the optimization. EAO will
* Create the LP/MIP (potentially needing much memory)
* solving the problem (potentially requiring much time)

In [18]:
tracemalloc.start() ## start memory tracking
out = eao.optimize(portf, timegrid, data)
# stopping memory tracking
print("Peak memory usage: ", f'{memory_used():,.0f}', "MB")
tracemalloc.stop()

Peak memory usage:  149 MB


We observe that solving this setup does not constitute an issue in terms of memory used or time required. 

Why is that?
* We used relatively simple assets with few restrictions. Therefore **little memory** is used despite a relatively large number of assets and many time steps
* Relatively **quick to solve**. The problem is an LP (no binary variables e.g. for start ups). Such problems can typically solved efficiently

Here, a few lines of code that allow us to check the nature of the optimization problem. Note that the above call "out = eao.optimize(portf, timegrid, data)" is a shortcut. We take it apart into the single steps to obtain the (1) optimization problem (2) do the optimization and (3) to extract the results in neat dataframes:

In [19]:
# (1) generate optim. problem
opt = portf.setup_optim_problem(data, timegrid)
### get info on problem type:
print(f"Is problem a MIP? {opt.is_MIP}")
#### to show the remaining steps for sake of completeness (not needed here)
# (2) optimize
# res = opt.optimize() ### commented out to save us some time
# (3) extract result into neat dataframes - also portfolio and optim problem needed)
#out = eao.io.extract_output(portf, opt, res)

Is problem a MIP? False


... so: no MIP and pretty easy to solve. Let's remember for later how to do the check

#### Storage: The most problematic asset when it comes to memory

Let us add a storage to the portfolio. It's just one asset more, but we will see that a storage requires much memory when the optimzation problem is set up

In [20]:

# storage
battery     = eao.assets.Storage(name = 'battery',
                                 nodes=      node_power,
                                 cap_out     = 1, 
                                 cap_in      = 1, 
                                 size        = 2, 
                                 eff_in      = 0.9, 
                                 no_simult_in_out = False,    # creates MIP of true! To be analyzed later
                                 block_size       = None)     # optimization over whole time horizon. To be analyzed later


In [21]:
tracemalloc.start() ## start memory tracking
battery.setup_optim_problem(prices = None, timegrid=eao.Timegrid(Start, End, freq=resolution))
# stopping memory tracking
print("Peak memory usage: ", f'{memory_used():,.0f}', "MB")
tracemalloc.stop()

Peak memory usage:  3,534 MB


Much memory is needed - even to generate the LP part of the storage only!

To provide some detail on the "why":
* in each point in time (every hour!) the storage is at least empty and at most full
* that means, the sum of all previous dispatches must be >= 0 and <= storage size

So we have two restrictions for each time step, involving all previous time steps. A densely populated matrix

**But wait: There's a very simple way around**: Is it realistic to assume the dispatch (e.g.) on Jan 1 affects (e.g.) my decision on Dec 31? In reality we can securely assume that a short-term storage is dispatched on a daily or weekly basis. This should even reflect trading reality better, than optimizing against the full time span. In the following we try it, using the parameter **block_size**.

In [22]:
battery.block_size = 'W' # weekly optimization. In each block, the battery goes back to start_level

tracemalloc.start() ## start memory tracking
battery.setup_optim_problem(prices = None, timegrid=eao.Timegrid(Start, End, freq=resolution))
# stopping memory tracking
print("Peak memory usage: ", f'{memory_used():,.0f}', "MB")
tracemalloc.stop()

Peak memory usage:  158 MB


**Recommendation:** Always utilize the block_size and set it to a low value. We found that daily ("d") is often a good choice for batteries or heat storage. Most available optimization tools utilize such simplification "under the hood". However, we believe it should be up to the user to decide whether or when to use it.

**Long-term storage:** An alternative for long-term storage such as seasonal storages for gas or large hydro storages: We can define different intrinsic frequencies for single assets. For a specific sample check out https://rivacon.github.io/EAO/samples/mixing_long_and_short_term/mixing_long_and_short_term.html

#### Solvers

For MIPs, solvers typically implement complex iterations using heuristics - and widely differ in efficiency. In EAO, for MIPs, we set the standard to the open source solver SCIP. For LPs we let CVXPY decide If you have a license for Gurobi or CPLEX, it may be worthwhile to switch. Simply set the parameter "solver = ..." when calling the optimization (see samples below).

**We are using CVXPY as interface to solvers**. See https://www.cvxpy.org/tutorial/solvers/index.html for available solvers. You may need to install solvers to your environment.

## Run-time - divide & conquer and LPs vs. MIPs

Let us now focus on run-time. While the size of a problem (roughly number of assets $\times$ number of time steps, storage block size) determines the problem size, its complexity is not necessarily linked to it.

A main lever for complexity is the question whether we're dealing with a **mixed integer problem (MIP) or a linear problem (LP)**:
* An LP only contains continuous variables. An LP is convex, with guaranteed convergence and efficient solution algorithms.
* A MIP contains "switch" variables such as start-up decisions. MIPs are not necessarily convex, no convergence guaranteed. Run-time may be long, even for smaller problems.

Unfortunately there's no easy way out and it turns out to be difficult knowing in advance, which problem is hard and which is not. In the coming sections we explore how to deal with complex problems.

#### Long simulations - splitting in chunks

Solving MIPs over long time frames is often not possible in one go (due to long run-time or memory usage). However, solving them in chunks of, say, weeks or days, is mostly doable. Particularly in power or gas setups, there is also no reason to believe that decisions early the year will influence decisions late in the year (e.g. starting a plant in January will hardly have an effect on whether the plant is running in December).

Splitting a problem into chunks manually would be cumbersome. Therefore we introduce "split_optimization" in EAO. This will cut the problem into chunks of user-defined length (e.g. weeks) and set together results automatically.

Attention in cases where we are using restrictions across a longer time frame - such as "minimum take". In this case, split-optimization will also divide down this restriction. E.g. a minimum take of 12 GWh across the year will give rise to 12 chunks of 1 GWh minimum take per month. In those case consider manual break-down or going for an LP (see below).

Let us define a sample portfolio: The main asset is the power plant (we're adding two of them with different parameters), where we use some of the more complex restrictions. Those are mainly start up costs and specific ramps for starting and shutting down as well as minimum down time. We add a battery with a maximum number of cycles per day to add complexity.

In [23]:
Start = dt.datetime(2024,1,1)
End   = dt.datetime(2025,1,1) # one month - for purpose of this demo a bit shorter 
resolution = 'h'  # hourly resolution

In [24]:
# define timegrid
timegrid = eao.assets.Timegrid(Start, End, freq=resolution)

### our sample here: optimized power plant
# define portfolio
node_power   = eao.assets.Node('node_power')
power_market = eao.assets.SimpleContract(name = 'market', nodes=node_power, price='price',
                                         min_cap = -110,    # we restrict grid connection so battery & plant cannot
                                         max_cap =  110 )   # run full load together
plant_1      = eao.assets.Plant(name    = 'plant_1',
                                nodes   = node_power,
                                extra_costs = 100,
                                min_cap               = 20.,
                                max_cap               = 100.,
                                ramp                  = 20,
                                start_costs           = 1000, 
                                min_downtime          = 8,
                                time_already_running  = 0,
                                time_already_off      = 1) 

plant_2      = eao.assets.Plant(name    = 'plant_2',
                                nodes   = node_power,
                                extra_costs           = 90,
                                min_cap               = 50.,
                                max_cap               = 90.,
                                ramp                  = 10,
                                start_costs           = 2000, 
                                min_downtime          = 10,
                                time_already_running  = 0,
                                time_already_off      = 1) 


battery     = eao.assets.Storage(name        = 'battery',
                                 nodes       = node_power,
                                 cap_out     = 50,  # MW
                                 cap_in      = 50,  # MW
                                 size        = 100, # MWh
                                 eff_in      = 0.9, # %
                                 start_level = 0.5, # MWh
                                 end_level   = 0.5, # MWh
                                 max_cycles_freq = 'd',
                                 max_cycles_no = 1,
                                 no_simult_in_out = True,   # creates MIP if true! If false, simultaneous
                                                            # charge/discharge for negative prices possible
                                 block_size       = 'd')    # daily optimization
portf = eao.portfolio.Portfolio([power_market, plant_1, plant_2, battery])

Let's try different variants: Optimization in one piece, split into weeks, relaxed to LP as well as different solvers.

In [25]:
start_time = time.time()
out = eao.optimize(portf, timegrid, data)
end_time = time.time()
minutes, seconds = divmod(end_time - start_time, 60)
print(f"Run-time: {int(minutes)} min {seconds:.0f} sec")
print(f"Total dispatch plant (no splitting using SCIP as EAO default solver): {out['dispatch'][['plant_1', 'plant_2']].sum().sum():,.0f} MWh")

Run-time: 10 min 14 sec
Total dispatch plant (no splitting using SCIP as EAO default solver): 210,996 MWh


In [26]:
start_time = time.time()
out = eao.optimize(portf, timegrid, data, solver = 'HIGHS')
end_time = time.time()
minutes, seconds = divmod(end_time - start_time, 60)
print(f"Run-time: {int(minutes)} min {seconds:.0f} sec")
print(f"Total dispatch plant (no splitting using HighS as solver): {out['dispatch'][['plant_1', 'plant_2']].sum().sum():,.0f} MWh")


Run-time: 11 min 56 sec
Total dispatch plant (no splitting using HighS as solver): 210,896 MWh


We see little difference between SCIP and HighS (we found both to be very efficient for EAO purposes). Note that plant output is very similar, but not identical (you will hardly achieve perfect accuracy in MIPs). It's worthwhile trying different solvers for your problems. Differences may be big. 

Particularly if you have a license for CPLEX, Gurobi or another commercial solver, it's worth trying it. Performance may be much better.

Now the run in weekly chunks:

In [27]:
start_time = time.time()
out = eao.optimize(portf, timegrid, data, split_interval_size='W')
end_time = time.time()
minutes, seconds = divmod(end_time - start_time, 60)
print(f"Run-time: {int(minutes)} min {seconds:.0f} sec")
print(f"Total dispatch plant (with splitting): {out['dispatch'][['plant_1', 'plant_2']].sum().sum():,.0f} MWh")

Run-time: 2 min 24 sec
Total dispatch plant (with splitting): 208,496 MWh


Check out the comparison of overall generation: the split optimization leads to similar results. A close look on hourly level may, of course, show greater differences in detail. For example: running through a weekend instead of shutting down for a Sunday night.

Note: In this illustration we chose examples with run-times that do not hurt too much in an interactive notebook. But note that for complex assets which require a MIP, split optimization easily makes a great difference (also between no convergence at all and quick results).

**Recommendation:** Use split optimization as far as possible for simulation across long time frames. Check your asset definitions and your analysis targets to guide you. Particularly where assets do not have long-range restrictions (such as minimum take or long-term storage), split optimization is a good choice.

#### Making the MIP an LP

A MIP is present if, for example, we have a binary decision such as whether to start oa plant that is tied to costs such as start-up costs. We utilize a boolean variables (0 = off, 1 = on) or (1 = is starting, 0 = is not starting).

A drastic simplification is to allow all values [0,1], making the problem an LP. Sometimes cutting down drastically on run-time. The main use-case, though, is for testing purpose in cases where a MIP is not feasible or does not converge at all. For example, decision variables such as for start-up to a minimum load may lead to cases where a small demand cannot be fulfilled (ill formulated problem). In those cases, a relaxed problem may help to identify mistakes or workarounds.

However, also for some use-cased, we believe this can be a valid approximation, particularly in long-term valuation.

Let us test, using the parameter make_soft_problem:

In [28]:
start_time = time.time()
out = eao.optimize(portf, timegrid, data, make_soft_problem=True)
end_time = time.time()
minutes, seconds = divmod(end_time - start_time, 60)
print(f"Run-time: {int(minutes)} min {seconds:.0f} sec")
print(f"Total dispatch plant (softened problem using default solver): {out['dispatch'][['plant_1', 'plant_2']].sum().sum():,.0f} MWh")

Run-time: 9 min 3 sec
Total dispatch plant (softened problem using default solver): 297,740 MWh


In our simple illustration, we see a limited impact run-time - yet in real-life problems differences may be much higher. The approximation appears somewhat ok, at least in terms of overall plant dispatch.

**Recommendation:** Use make_soft_problem with care. When a problem does not converge or when it is not feasible (it easily happens that restrictions cannot be met due to errors in the setup), start trying the soft version. When using for a productive purposes, do thorough tests. Start-up processes, for example, will not be modelled correctly.

#### 